# Packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statistics import mean
import plotly.graph_objects as go
import pylab
import warnings
from scipy.stats import sem
from datetime import datetime
#from termcolor import colored
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold, KFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [2]:
pip install plotly

     |████████████████████████████████| 15.2 MB 470 kB/s eta 0:00:01
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


Loading data

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
df=pd.read_csv('Akkumulated_tennis_PR.csv')

df=df.drop(['Unnamed: 0.1',
 'Unnamed: 0',
 'P1Ace',
 'P2Ace',
 'P1Winner',
 'P2Winner',
 'P1DoubleFault',
 'P2DoubleFault',
 'P1UnfErr',
 'P2UnfErr',
 'P1NetPoint',
 'P2NetPoint',
 'P1NetPointWon',
 'P2NetPointWon',
 'P1BreakPoint',
 'P2BreakPoint',
 'P1BreakPointWon',
 'P2BreakPointWon',
 'P1BreakPointMissed',
 'P2BreakPointMissed',
 'ServeIndicator',
 'ServeNumber',
 'WinnerType',
 'WinnerShotType',
 'P1DistanceRun',
 'P2DistanceRun',
 'RallyCount','P1_ServeWidth_0',
 'P1_ServeWidth_B',
 'P1_ServeWidth_BC',
 'P1_ServeWidth_BW',
 'P1_ServeWidth_C',
 'P1_ServeWidth_DoubleFault',
 'P1_ServeWidth_W',
 'P1_ServeDepth_0',
 'P1_ServeDepth_CTL',
 'P1_ServeDepth_DoubleFault',
 'P1_ServeDepth_NCTL',
 'P1_ReturnDepth_0',
 'P1_ReturnDepth_D',
 'P1_ReturnDepth_DoubleFault',
 'P1_ReturnDepth_ND',
 'P1_ReturnDepth_ServeAce',
 'P1_ReturnDepth_Service box',
 'P2_ServeWidth_0',
 'P2_ServeWidth_B',
 'P2_ServeWidth_BC',
 'P2_ServeWidth_BW',
 'P2_ServeWidth_C',
 'P2_ServeWidth_DoubleFault',
 'P2_ServeWidth_W',
 'P2_ServeDepth_0',
 'P2_ServeDepth_CTL',
 'P2_ServeDepth_DoubleFault',
 'P2_ServeDepth_NCTL',
 'P2_ReturnDepth_0',
 'P2_ReturnDepth_D',
 'P2_ReturnDepth_DoubleFault',
 'P2_ReturnDepth_ND',
 'P2_ReturnDepth_ServeAce',
 'P2_ReturnDepth_Service box',
 'ElapsedTime',
 'ServeWidth',
 'ServeDepth',
 'P1Momentum',
 'P2Momentum',
 'ReturnDepth',
 'datetime',
 'year',
 'slam',
 'match_num',
 'player1',
 'player2',
 'tourney_name',
 'tourney_date',
 'winner_name',
 'winner_hand',
 'loser_name',
 'loser_hand'],axis=1)

Making rank for p1 and p2

In [ ]:
df.loc[df['match_id'] == "2020-usopen-1228", 'loser_rank'] = 501

In [ ]:
p1_rank = []
p2_rank = []
for i in range(len(df)):
    if df.player1.iloc[i] == df.winner_name.iloc[i]:
        p1_rank.append(df.winner_rank.iloc[i])
        p2_rank.append(df.loser_rank.iloc[i])
    if df.player1.iloc[i] == df.loser_name.iloc[i]:
        p1_rank.append(df.loser_rank.iloc[i])
        p2_rank.append(df.winner_rank.iloc[i])
df['P1Rank'] = p1_rank
df['P2Rank'] = p2_rank

Making total elapsed time in seconds

In [ ]:
total_time=[]
for i in range(len(df.match_id.unique())):
    total_time = np.append(total_time, np.insert(df[df['match_id'] == df.match_id.unique()[i]].Point_lenght_sec.cumsum().values[:-1],0,0))
df['Total_time']=total_time

Creating a tiebreak variable

Columns to be shifted

In [ ]:
to_be_shifted=[
    ('P1GamesWon','P2GamesWon'),
    ('P1Score','P2Score'),
    ('P1PointsWon','P2PointsWon'),
    ('P1_ServeWidth_B_A','P2_ServeWidth_B_A'),
    ('P1_ServeWidth_BC_A','P2_ServeWidth_BC_A'),
    ('P1_ServeWidth_BW_A','P2_ServeWidth_BW_A'),
    ('P1_ServeWidth_C_A','P2_ServeWidth_C_A'),
    ('P1_ServeWidth_W_A','P2_ServeWidth_W_A'),
    ('P1_ServeDepth_CTL_A','P2_ServeDepth_CTL_A'),
    ('P1_ServeDepth_NCTL_A','P2_ServeDepth_NCTL_A'),
    ('P1_ReturnDepth_D_A','P2_ReturnDepth_D_A'),
    ('P1_ReturnDepth_ND_A','P2_ReturnDepth_ND_A'),
    ('P1AceA','P2AceA'),
    ('P1WinnerA','P2WinnerA'),
    ('P1DoubleFaultA','P2DoubleFaultA'),
    ('P1UnfErrA','P2UnfErrA'),
    ('P1NetPointA','P2NetPointA'),
    ('P1NetPointWonA','P2NetPointWonA'),
    ('P1BreakPointA','P2BreakPointA'),
    ('P1BreakPointWonA','P2BreakPointWonA'),
    ('P1BreakPointMissedA','P2BreakPointMissedA'),
    ('P1DistanceRunA','P1DistanceRunA'),
    ('P1SetsWon','P2SetsWon'),
    ('player1','player2'),
    ('P1NetPoint','P2NetPoint'),
    ('P1Rank','P2Rank')
    ]

Changing pointwinenr to binary encoding 0 = p1, 1= p2

In [ ]:
Server = {1: 0,2: 1}
df.PointServer = [Server[item] for item in df.PointServer]
df.PointWinner = [Server[item] for item in df.PointWinner]
df.GameWinnerA = [Server[item] for item in df.GameWinnerA]
df.SetWinnerA = [Server[item] for item in df.SetWinnerA]

Surface attribute, 0 = US open (Turf), 1 = Wimbeldon grass

In [ ]:
surface=[]
for i in range(len(df)):
    if df.match_id.iloc[i][5] == "u":
        surface.append(0)
    else:
        surface.append(1)
df['Surface']=surface

TieBreak

In [ ]:
tie=[]
for i in range(len(df)):
    if (df.GameNo.iloc[i]>12) and (df.Surface.iloc[i]==0):
        tie.append(1)
    elif (df.GameNo.iloc[i] == 25) and (df.Surface.iloc[i]==1) and (df.year.iloc[i]>2018):
        tie.append(1)
    elif (df.GameNo.iloc[i] > 12) and (df.Surface.iloc[i]==1) and (df.SetNo.iloc[i] < 5):
        tie.append(1)
    else:
        tie.append(0)
df['Tiebreak']=tie

Editing point number mistakes in data

In [ ]:
for i in range(len(df)):
    if pd.isna(df.iloc[i]['PointNumber']):
        df['PointNumber'].iloc[i]=df['PointNumber'].iloc[i+1]-1

Editing NA value for loser rank in a single match

Making unique time stamps for what point number in the game it is

In [ ]:
game_timestamp=[]

k=0
for i in range(len(df)):
    if df.GameNo.iloc[i] != df.GameNo.iloc[i-1]:
        k=1
        game_timestamp.append(k)
    if df.GameNo.iloc[i] == df.GameNo.iloc[i-1]:
        k=k+1
        game_timestamp.append(k)
df['Game_timestamp']=game_timestamp

Attributes for predictions

In [ ]:
Game_test_columns=['SetNo',
                   'P1GamesWon',
                   'P2GamesWon',
                   'GameNo',
                   'PointNumber',
                   'Tiebreak',
                   'P1Score',
                   'P2Score',
                   'P1PointsWon',
                   'P2PointsWon',
                   'P1_ServeWidth_B_A',
                   'P1_ServeWidth_BC_A',
                   'P1_ServeWidth_BW_A',
                   'P1_ServeWidth_C_A',
                   'P1_ServeWidth_W_A',
                   'P1_ServeDepth_CTL_A',
                   'P1_ServeDepth_NCTL_A',
                   'P1_ReturnDepth_D_A',
                   'P1_ReturnDepth_ND_A',
                   'P2_ServeWidth_B_A',
                   'P2_ServeWidth_BC_A',
                   'P2_ServeWidth_BW_A',
                   'P2_ServeWidth_C_A',
                   'P2_ServeWidth_W_A',
                   'P2_ServeDepth_CTL_A',
                   'P2_ServeDepth_NCTL_A',
                   'P2_ReturnDepth_D_A',
                   'P2_ReturnDepth_ND_A',
                   'P1AceA',
                   'P2AceA',
                   'P1WinnerA',
                   'P2WinnerA',
                   'P1DoubleFaultA',
                   'P2DoubleFaultA',
                   'P1UnfErrA',
                   'P2UnfErrA',
                   'P1NetPointA',
                   'P2NetPointA',
                   'P1NetPointWonA',
                   'P2NetPointWonA',
                   'P1BreakPointA',
                   'P2BreakPointA',
                   'P1BreakPointWonA',
                   'P2BreakPointWonA',
                   'P1BreakPointMissedA',
                   'P2BreakPointMissedA',
                   'P1DistanceRunA',
                   'P2DistanceRunA',
                   'RallyCountA',
                   'P1SetsWon',
                   'P2SetsWon',
                   'Game_timestamp',
                   'P1Rank',
                   'P2Rank',
                   'Total_time',
                   'Surface']

k=0
for i in range(len(df)-1):
    if df2['P2GamesWon'].loc[i] > df2.P2GamesWon.loc[i+1] and df2['P2GamesWon'].loc[i+1] != 0:
        k=k+1
        print(i)

In [ ]:
#df2[57924:57928]  Der er en fejl i P2GamesWOn her

Shifting columns

In [ ]:
df2=df.copy()
for i in range(len(df)):
    if df['PointServer'].loc[i] == 1:
        for j in range(len(to_be_shifted)):
            df.at[i,to_be_shifted[j][0]] = df2[to_be_shifted[j][1]].loc[i]
            df.at[i,to_be_shifted[j][1]] = df2[to_be_shifted[j][0]].loc[i]
        if df.PointWinner.loc[i] == 0:
            df.at[i,"PointWinner"] = 1
        elif df.PointWinner.loc[i] == 1:
            df.at[i,"PointWinner"] = 0
            
        if df.GameWinnerA.loc[i] == 0:
            df.at[i,"GameWinnerA"]=1
        elif df.GameWinnerA.loc[i] == 1: 
            df.at[i,"GameWinnerA"] = 0

        if df.SetWinnerA.loc[i] == 0:
            df.at[i,"SetWinnerA"]=1
        elif df.SetWinnerA.loc[i] == 1: 
            df.at[i,"SetWinnerA"] = 0
            
    

# Functions

In [ ]:
def metrics(Y_test,y_pred,x_val,model):
    val_pred=y_pred
    x_val=x_val
    model=model
    auc=round(accuracy_score(Y_test,val_pred),4)
    f_score=round(f1_score(Y_test, val_pred),4)
    P_score=round(precision_score(Y_test,val_pred),4)
    recall=round(recall_score(Y_test, val_pred),4)
    
    ns_probs = [0 for _ in range(len(Y_test))]
    lr_probs = model.predict_proba(x_val)
    lr_probs = lr_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(Y_test, ns_probs)
    lr_auc = round(roc_auc_score(Y_test, lr_probs),4)
    return [auc,f_score,P_score,recall,lr_auc]
def averages1(name,metrics):
    
    res = [round(sum(x) / len(x),3) for x in zip(*metrics)]
    print(f"{name} Accuracy score: {res[0]*100}, f1 score: {res[1]}, Precision: {res[2]}, Recall: {res[3]}, ROC-AUC: {res[4]}\n")
    return

def averages(metrics):
    
    res = [round(sum(x) / len(x),3) for x in zip(*metrics)]
    #print(f"{name} Accuracy score: {res[0]*100}, f1 score: {res[1]}, Precision: {res[2]}, Recall: {res[3]}, ROC-AUC: {res[4]}")
    return res

import random
def data_split(df):
    matches=list(df.match_id.unique())
    Val_games=random.sample(matches,round(len(matches)/10))
    df_val=df[df['match_id'].isin(Val_games)]
    df=df[~df['match_id'].isin(Val_games)]
    return df,df_val


def plot_feature_importance(importance,names,model_type):

    #Create arrays from feature importance and feature names
    feature_importance = importance
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_df = fi_df[fi_df.feature_importance > 0.01]
    

    #Define size of bar plot
    plt.figure(figsize=(8,6))
    plt.tight_layout()
    plt.yticks(fontsize=7)
    #Plot Searborn bar chart
    ax=sns.barplot(y='feature_names',x='feature_importance',data=fi_df,palette="rocket")
    ax.set_xlabel('feature_importance')
    #plt.xticks(rotation = 90)
    
    #Add chart labels
    plt.title(model_type + ' FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE > 0.01')
    plt.ylabel('FEATURE NAMES')


def evaluation_time(y_val,y_pred,model,x_val):
    plt.style.use('ggplot')
    cm = confusion_matrix(y_val, (val_pred))
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax,fmt='g',cmap='rocket'); #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted Winner');ax.set_ylabel('Actual Winner'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['Server', 'Returner']); ax.yaxis.set_ticklabels(['Server', 'Returner'], rotation=360);
    plt.show()

    f_score=f1_score(y_val, val_pred, average=None)
    P_score=precision_score(y_val,val_pred,average=None)
    recall=recall_score(y_val, val_pred, average=None)


    ns_probs = [0 for _ in range(len(y_val))]
    lr_probs = model.predict_proba(x_val)
    lr_probs = lr_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(y_val, ns_probs)
    lr_auc = roc_auc_score(y_val, lr_probs)
    # summarize scores
    print(f"Test set accuracy score {(np.round(accuracy_score(y_pred,y_val),3)*100)}%")
    print(f"F1 score {np.round(f_score,3)}")
    print(f"Recall score {np.round(recall,3)}")
    print(f"Precision score {np.round(P_score,3)}")
    print('Model: ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_val, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_val, lr_probs)
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Base')
    plt.plot(lr_fpr, lr_tpr, marker='.', label='XGBoost')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()

def evaluation(y_val,y_pred,model_xg,x_val):
    plt.style.use('ggplot')
    cm = confusion_matrix(y_val, (val_pred))
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax,fmt='g',cmap='rocket'); #annot=True to annotate cells

    ax.set_xlabel('Predcited Winner');ax.set_ylabel('Actual Winner'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['Server', 'Returner']); ax.yaxis.set_ticklabels(['Server', 'Returner'], rotation=360);
    plt.show()

    f_score=f1_score(y_val, val_pred, average=None)
    P_score=precision_score(y_val,val_pred,average=None)
    recall=recall_score(y_val, val_pred, average=None)


    ns_probs = [0 for _ in range(len(y_val))]
    lr_probs = model_xg.predict_proba(x_val)
    lr_probs = lr_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(y_val, ns_probs)
    lr_auc = roc_auc_score(y_val, lr_probs)
    # summarize scores
    print(f"Test set accuracy score {np.round(val_scoreXG*100,3)}%")
    print(f"F1 score {np.round(f_score,3)}")
    print(f"Recall score {np.round(recall,3)}")
    print(f"Precision score {np.round(P_score,3)}")
    print('Xgboost: ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_val, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_val, lr_probs)
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Base')
    plt.plot(lr_fpr, lr_tpr, marker='.', label='XGBoost')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()
    
    results = model_xg.evals_result()
    epochs = len(results['validation_0']['error'])
    x_axis = range(0, epochs)
    # plot log loss
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
    ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
    ax.legend()
    plt.ylabel('Log Loss')
    plt.title('XGBoost Log Loss')
    plt.show()
    # plot classification error
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['error'], label='Train')
    ax.plot(x_axis, results['validation_1']['error'], label='Test')
    ax.legend()
    plt.ylabel('Classification Error')
    plt.title('XGBoost Classification Error')
    plt.show()
    return

def errors(values):
    errors=[]
    for i in range(len(values)):
        errors.append(sem(values[i])*100)
    return errors


def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
        
def sample(x,y):
    xx=np.unique(y,return_counts=True)
    xx=xx[1][0]/xx[1][1]
    if xx < 4:
        return x,y
    if xx > 4:
        oversample = SMOTE(0.5)
        x, y = oversample.fit_resample(x, y)
        return x,y

Creating Train and test set data

In [ ]:
df_copy=df.copy()
df,df_val=data_split(df)

# Point Winner

### Initial model - RFC, XG, ADA, LOGREG, 10fold

In [ ]:
y=df['PointWinner'].values
x=df[Game_test_columns]
labels=list(x.columns)
x=x.values
scaler = StandardScaler()
x=scaler.fit_transform(x)
warnings.filterwarnings('ignore')

k=0
for i in range(len(df)):
    if df['PointWinner'].iloc[i] == df['PointServer'].iloc[i]:
        k=k+1
baseline=(k/len(df))*100


folds = 10
kf = KFold(n_splits=folds, shuffle=True)
RFCScore = []
adaScore = []
XGBScore = []
LRScore = []
FI_rfc = []
FI_ada = []
FI_XGB = []
metrics_models=[]

i=0
for train_index, test_index in kf.split(x):

    X_train, X_test = x[train_index], x[test_index]
    Y_train, Y_test = y[train_index], y[test_index]

    
    #RF
    
    RFC = RandomForestClassifier(max_depth=6, random_state=0)
    predRFC = RFC.fit(X_train,Y_train)
    y_predRFC = predRFC.predict(X_test)
    #scoreRFC = accuracy_score(y_predRFC, Y_test)
    RFCScore.append(metrics(Y_test,y_predRFC,X_test,RFC))
    FI_rfc.append(RFC.feature_importances_)
    
    #ADA

    #ada = AdaBoostClassifier(base_estimator = RandomForestClassifier(max_depth=2, random_state=0))
    #predADA = ada.fit(X_train, Y_train)
    #y_predADA = predADA.predict(X_test)
    #scoreADA = accuracy_score(y_predADA, Y_test)
    #adaScore.append(metrics(Y_test,y_predADA,X_test,ada))
    #FI_ada.append(ada.feature_importances_)
    
    
    #XG
    model_xg = XGBClassifier(objective ='binary:logistic')
    model_xg.fit(X_train, Y_train);
    y_pred_xg = model_xg.predict(X_test);
    scoreXG = accuracy_score(y_pred_xg, Y_test)
    XGBScore.append(metrics(Y_test,y_pred_xg,X_test,model_xg))
    FI_XGB.append(model_xg.feature_importances_)
    
    #LR
    logreg = LogisticRegression()
    logreg.fit(X_train, Y_train)
    y_pred_lr=logreg.predict(X_test)
    LRScore.append(metrics(Y_test,y_pred_lr,X_test,logreg))
    #FI_LR.append(logreg.feature_importances_)
    

    i+=1
    print(f"Processing fold {i}")
warnings.filterwarnings('default')    
#rf_scores.append(np.mean(RFCScore)*100)
#xg_scores.append(np.mean(XGBScore)*100)
print("\nModel Accuracy")


print(f"Baseline 65%")
averages1("RFC",RFCScore)
averages1("XGBoost",XGBScore)
averages1("Logistic Regression",LRScore)
#averages1("AdaBoost",adaScore)

print("\n Model feature impact > 0.01")
#plot_feature_importance(list(map(mean, zip(*FI_rfc))),labels,"Random Forrest")
#plot_feature_importance(list(map(mean, zip(*FI_XGB))),labels,"XG")

#### Defining grid for XGBoost for tuning

In [ ]:
#Values
params={
 "min_child_weight" : [ 1, 3, 5, 7, 9 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.6, 0.9, 1.2 , 1.5 ],
 "subsample": np.arange(0.5, 1.0, 0.11).tolist(),
 "max_depth"        : [ 4, 5, 6, 8, 10, 12],
 "learning_rate"    : [0.01, 0.05, 0.08, 0.1],
 "scale_pos_weight":np.arange(1, 1.5, 0.1).tolist()
}

### Hyperparameter tunining pointwinner

In [ ]:
#Datasplit
X_train, X_test, y_train, y_test = train_test_split(df[Game_test_columns],df['PointWinner'].values, test_size=0.20, random_state=42)

#Feature scaling
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#Model for tuning
classifier=XGBClassifier(objective='binary:logistic')


#RandomSearch
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=10,scoring='f1',n_jobs=-1,cv=10,verbose=3)
start_time = timer(None)
#random_search.fit(X_train,y_train)
timer(start_time) 

#random_search.best_params_
#Grid_Search
#grid_search=GridSearchCV(classifier,param_grid=params,scoring='f1',n_jobs=-1,cv=10)
#start_time = timer(None)
#grid_search.fit(X_train,y_train)
#timer(start_time)


min_child_weight=3,
                         gamma=0.2,
                         colsample_bytree=0.3,
                         sub_sample=0.94,
                          max_depth=5,
                         learning_rate = 0.05,
                         scale_pos_weight=1.1,
                         verbosity=0,
                         use_label_encoder=False,

In [ ]:
param={'n_estimators': 1500,
 'min_samples_split': 5,
 'min_samples_leaf': 15,
 'max_features': 'accuracy',
 'max_depth': 10,
 'bootstrap': True}

In [ ]:
y=df[df.ServeNumber==1]['PointWinner'].values
x=df[df.ServeNumber==1][Game_test_columns]
y_val=df_val[df_val.ServeNumber==1]['PointWinner'].values
x_val=df_val[df_val.ServeNumber==1][Game_test_columns].values
labels=Game_test_columns

#undersample = RandomUnderSampler(sampling_strategy=0.7)
#x, y = undersample.fit_resample(x, y)
#print(x.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
x_val=scaler.transform(x_val);


model_xg = XGBClassifier(objective ='binary:logistic',min_child_weight=1,
                         gamma=0.0,
                         colsample_bytree=0.2,
                         sub_sample=0.8,
                          max_depth=5,
                         learning_rate = 0.08,
                         scale_pos_weight=1.6,
                         verbosity=0,
                         use_label_encoder=False,)

model_xg.fit(X_train, y_train);
y_pred_xg = model_xg.predict(X_test);
scoreXG = accuracy_score(y_pred_xg, y_test)
print(f"Model training score {round(scoreXG,3)*100}")
val_pred=model_xg.predict(x_val)
val_scoreXG = accuracy_score(val_pred, y_val)
evaluation_time(y_val,val_pred,model_xg,x_val)
plot_feature_importance(model_xg.feature_importances_,labels,"XGBoost First serve")
plt.savefig("Pointwinner_S1_Features - XGboost",bbox_inches='tight',dpi=300)

In [ ]:
y=df[(df.ServeNumber==2) | (df.ServeNumber==0)]['PointWinner'].values
x=df[(df.ServeNumber==2) | (df.ServeNumber==0)][Game_test_columns]
y_val=df_val[(df_val.ServeNumber==2) | (df_val.ServeNumber==0)]['PointWinner'].values
x_val=df_val[(df_val.ServeNumber==2) | (df_val.ServeNumber==0)][Game_test_columns].values
labels=Game_test_columns

#undersample = RandomUnderSampler(sampling_strategy=0.9)
#x, y = undersample.fit_resample(x, y)
#print(x.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
x_val=scaler.transform(x_val);
model_xg = XGBClassifier(objective ='binary:logistic',min_child_weight=1,
                         gamma=0.2,
                         colsample_bytree=0.6,
                         sub_sample=1,
                          max_depth=6,
                         learning_rate = 0.08,
                         scale_pos_weight=1.0,
                         verbosity=0,
                         use_label_encoder=False,)


model_xg.fit(X_train, y_train);
y_pred_xg = model_xg.predict(X_test);
scoreXG = accuracy_score(y_pred_xg, y_test)
print(f"Model training score {round(scoreXG,3)*100}")

val_pred=model_xg.predict(x_val)
val_scoreXG = accuracy_score(val_pred, y_val)
evaluation_time(y_val,val_pred,model_xg,x_val)
plot_feature_importance(model_xg.feature_importances_,labels,"XG Second Serve ")
plt.savefig("Pointwinner_S2_Features - XGboost",bbox_inches='tight',dpi=300)

 # Serve study


In [ ]:
Game_test_columns=['SetNo',
                   'P1GamesWon',
                   'P2GamesWon',
                   'GameNo',
                   'PointNumber',
                   #'PointServer',
                   'Tiebreak',
                   'P1Score',
                   'P2Score',
                   'P1PointsWon',
                   'P2PointsWon',
                   'Point_lenght_sec',
                   'P1_ServeWidth_B_A',
                   'P1_ServeWidth_BC_A',
                   'P1_ServeWidth_BW_A',
                   'P1_ServeWidth_C_A',
                   'P1_ServeWidth_W_A',
                   'P1_ServeDepth_CTL_A',
                   'P1_ServeDepth_NCTL_A',
                   'P1_ReturnDepth_D_A',
                   'P1_ReturnDepth_ND_A',
                   'P2_ServeWidth_B_A',
                   'P2_ServeWidth_BC_A',
                   'P2_ServeWidth_BW_A',
                   'P2_ServeWidth_C_A',
                   'P2_ServeWidth_W_A',
                   'P2_ServeDepth_CTL_A',
                   'P2_ServeDepth_NCTL_A',
                   'P2_ReturnDepth_D_A',
                   'P2_ReturnDepth_ND_A',
                   'P1AceA',
                   'P2AceA',
                   'P1WinnerA',
                   'P2WinnerA',
                   'P1DoubleFaultA',
                   'P2DoubleFaultA',
                   'P1UnfErrA',
                   'P2UnfErrA',
                   'P1NetPointA',
                   'P2NetPointA',
                   'P1NetPointWonA',
                   'P2NetPointWonA',
                   'P1BreakPointA',
                   'P2BreakPointA',
                   'P1BreakPointWonA',
                   'P2BreakPointWonA',
                   'P1BreakPointMissedA',
                   'P2BreakPointMissedA',
                   'P1DistanceRunA',
                   'P2DistanceRunA',
                   'RallyCountA',
                   'P1SetsWon',
                   'P2SetsWon',
                   'Game_timestamp',
                   'P1Rank',
                   'P2Rank',
                   'Total_time',
                   'Surface',
                    'ServeWidth',
                    'ServeDepth',
                    'ReturnDepth']

In [ ]:
df,df_val=data_split(df_copy)
df=df[(df.ReturnDepth != "ServeAce") & (df.ServeDepth != "DoubleFault")]
df_val=df_val[(df_val.ReturnDepth != "ServeAce") & (df_val.ServeDepth != "DoubleFault")]

In [ ]:
print(f"Baseline Serve 1: {round(df[df.ServeNumber==1].PointWinner.value_counts()[0]/len(df[df.ServeNumber==1]),3)}")
print(f"Baseline Serve 2: {round(df[df.ServeNumber==2].PointWinner.value_counts()[0]/len(df[df.ServeNumber==2]),3)}")

In [ ]:
y=df[df.ServeNumber==1]['PointWinner'].values
x=pd.get_dummies(df[df.ServeNumber==1][Game_test_columns])
labels=x.columns
labels=x.columns
scaler = StandardScaler()
x=scaler.fit_transform(x)
warnings.filterwarnings('ignore')


folds = 10
kf = KFold(n_splits=folds, shuffle=True)
RFCScore = []
adaScore = []
XGBScore = []
LRScore = []
FI_rfc = []
FI_ada = []
FI_XGB = []
metrics_models=[]

i=0
for train_index, test_index in kf.split(x):

    X_train, X_test = x[train_index], x[test_index]
    Y_train, Y_test = y[train_index], y[test_index]

    
    #RF
    
    RFC = RandomForestClassifier(max_depth=6, random_state=0)
    predRFC = RFC.fit(X_train,Y_train)
    y_predRFC = predRFC.predict(X_test)
    #scoreRFC = accuracy_score(y_predRFC, Y_test)
    RFCScore.append(metrics(Y_test,y_predRFC,X_test,RFC))
    FI_rfc.append(RFC.feature_importances_)
    
    #ADA

    ada = AdaBoostClassifier(base_estimator = RandomForestClassifier(max_depth=2, random_state=0))
    predADA = ada.fit(X_train, Y_train)
    y_predADA = predADA.predict(X_test)
    scoreADA = accuracy_score(y_predADA, Y_test)
    adaScore.append(metrics(Y_test,y_predADA,X_test,ada))
    FI_ada.append(ada.feature_importances_)
    
    
    #XG
    model_xg = XGBClassifier(objective ='binary:logistic',min_child_weight=5,gamma=0.2,colsample_bytree=0.9,
                                     max_depth=6,learning_rate = 0.01,verbosity=0,use_label_encoder=False)
    model_xg.fit(X_train, Y_train);
    y_pred_xg = model_xg.predict(X_test);
    scoreXG = accuracy_score(y_pred_xg, Y_test)
    XGBScore.append(metrics(Y_test,y_pred_xg,X_test,model_xg))
    FI_XGB.append(model_xg.feature_importances_)
    
    #LR
    logreg = LogisticRegression()
    logreg.fit(X_train, Y_train)
    y_pred_lr=logreg.predict(X_test)
    LRScore.append(metrics(Y_test,y_pred_lr,X_test,logreg))
    #FI_LR.append(logreg.feature_importances_)
    

    i+=1
    print(f"Processing fold {i}")
warnings.filterwarnings('default')    
#rf_scores.append(np.mean(RFCScore)*100)
#xg_scores.append(np.mean(XGBScore)*100)
print("\nModel Accuracy")


averages1("RFC",RFCScore)
averages1("XGBoost",XGBScore)
averages1("Logistic Regression",LRScore)
averages1("AdaBoost",adaScore)

print("\n Model feature impact > 0.01")
#plot_feature_importance(list(map(mean, zip(*FI_rfc))),labels,"Random Forrest")
#plot_feature_importance(list(map(mean, zip(*FI_XGB))),labels,"XG")

In [ ]:
y=df[(df.ServeNumber==2) | (df.ServeNumber==0)]['PointWinner'].values
x=pd.get_dummies(df[(df.ServeNumber==2) | (df.ServeNumber==0)][Game_test_columns])
labels=list(x.columns)
x=x.values
scaler = StandardScaler()
x=scaler.fit_transform(x)
warnings.filterwarnings('ignore')

k=0
for i in range(len(df)):
    if df['PointWinner'].iloc[i] == df['PointServer'].iloc[i]:
        k=k+1
baseline=(k/len(df))*100


folds = 10
kf = KFold(n_splits=folds, shuffle=True)
RFCScore = []
adaScore = []
XGBScore = []
LRScore = []
FI_rfc = []
FI_ada = []
FI_XGB = []
metrics_models=[]

i=0
for train_index, test_index in kf.split(x):

    X_train, X_test = x[train_index], x[test_index]
    Y_train, Y_test = y[train_index], y[test_index]

    
    #RF
    
    RFC = RandomForestClassifier(max_depth=6, random_state=0)
    predRFC = RFC.fit(X_train,Y_train)
    y_predRFC = predRFC.predict(X_test)
    #scoreRFC = accuracy_score(y_predRFC, Y_test)
    RFCScore.append(metrics(Y_test,y_predRFC,X_test,RFC))
    FI_rfc.append(RFC.feature_importances_)
    
    #ADA

    ada = AdaBoostClassifier(base_estimator = RandomForestClassifier(max_depth=2, random_state=0))
    predADA = ada.fit(X_train, Y_train)
    y_predADA = predADA.predict(X_test)
    scoreADA = accuracy_score(y_predADA, Y_test)
    adaScore.append(metrics(Y_test,y_predADA,X_test,ada))
    FI_ada.append(ada.feature_importances_)
    
    
    #XG
    model_xg = XGBClassifier(objective ='binary:logistic',min_child_weight=5,gamma=0.2,colsample_bytree=0.9,
                                     max_depth=6,learning_rate = 0.01,verbosity=0,use_label_encoder=False)
    model_xg.fit(X_train, Y_train);
    y_pred_xg = model_xg.predict(X_test);
    scoreXG = accuracy_score(y_pred_xg, Y_test)
    XGBScore.append(metrics(Y_test,y_pred_xg,X_test,model_xg))
    FI_XGB.append(model_xg.feature_importances_)
    
    #LR
    logreg = LogisticRegression()
    logreg.fit(X_train, Y_train)
    y_pred_lr=logreg.predict(X_test)
    LRScore.append(metrics(Y_test,y_pred_lr,X_test,logreg))
    #FI_LR.append(logreg.feature_importances_)
    

    i+=1
    print(f"Processing fold {i}")
warnings.filterwarnings('default')    
#rf_scores.append(np.mean(RFCScore)*100)
#xg_scores.append(np.mean(XGBScore)*100)
print("\nModel Accuracy")


print(f"Baseline 65%")
averages1("RFC",RFCScore)
averages1("XGBoost",XGBScore)
averages1("Logistic Regression",LRScore)
averages1("AdaBoost",adaScore)

print("\n Model feature impact > 0.01")
#plot_feature_importance(list(map(mean, zip(*FI_rfc))),labels,"Random Forrest")
#plot_feature_importance(list(map(mean, zip(*FI_XGB))),labels,"XG")

# Optimere pointwinner serve 1, 2.0

In [ ]:
#Datasplit
X_train, X_test, y_train, y_test = train_test_split(pd.get_dummies(df[df.ServeNumber==1][Game_test_columns]),df[df.ServeNumber==1]['PointWinner'].values, test_size=0.20, random_state=42)

#Feature scaling
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#Model for tuning
classifier=XGBClassifier(objective='binary:logistic')


#RandomSearch
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=25,scoring='accuracy',n_jobs=-1,cv=10,verbose=3)
start_time = timer(None)
random_search.fit(X_train,y_train)
timer(start_time) 

random_search.best_params_
#Grid_Search
#grid_search=GridSearchCV(classifier,param_grid=params,scoring='f1',n_jobs=-1,cv=10)
#start_time = timer(None)
#grid_search.fit(X_train,y_train)
#timer(start_time)



In [ ]:
y=df[df.ServeNumber==1]['PointWinner'].values
x=pd.get_dummies(df[df.ServeNumber==1][Game_test_columns])
y_val=df_val[df_val.ServeNumber==1]['PointWinner'].values
x_val=pd.get_dummies(df_val[df_val.ServeNumber==1][Game_test_columns])
labels=x.columns

#undersample = RandomUnderSampler(sampling_strategy=0.7)
#x, y = undersample.fit_resample(x, y)
#print(x.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
x_val=scaler.transform(x_val);


model_xg = XGBClassifier(objective ='binary:logistic',param=random_search.best_params_)

model_xg.fit(X_train, y_train);
y_pred_xg = model_xg.predict(X_test);
scoreXG = accuracy_score(y_pred_xg, y_test)
print(f"Model training score {round(scoreXG,3)*100}")
val_pred=model_xg.predict(x_val)
val_scoreXG = accuracy_score(val_pred, y_val)
evaluation_time(y_val,val_pred,model_xg,x_val)
plot_feature_importance(model_xg.feature_importances_,labels,"XGBoost Revised First serve")
plt.savefig("Pointwinner_S1_V2_Features - XGboost",bbox_inches='tight',dpi=300)

# Random search modele 2.2

In [ ]:
#Datasplit
X_train, X_test, y_train, y_test = train_test_split(pd.get_dummies(df[(df.ServeNumber==2) | (df.ServeNumber==0)][Game_test_columns]),df[(df.ServeNumber==2) | (df.ServeNumber==0)]['PointWinner'].values, test_size=0.20, random_state=42)

#Feature scaling
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#Model for tuning
classifier=XGBClassifier(objective='binary:logistic')


#RandomSearch
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=25,scoring='accuracy',n_jobs=-1,cv=10,verbose=3)
start_time = timer(None)
random_search.fit(X_train,y_train)
timer(start_time) 

random_search.best_params_
#Grid_Search
#grid_search=GridSearchCV(classifier,param_grid=params,scoring='f1',n_jobs=-1,cv=10)
#start_time = timer(None)
#grid_search.fit(X_train,y_train)
#timer(start_time)




In [ ]:
y=df[(df.ServeNumber==2) | (df.ServeNumber==0)]['PointWinner'].values
x=pd.get_dummies(df[(df.ServeNumber==2) | (df.ServeNumber==0)][Game_test_columns])
y_val=df_val[(df_val.ServeNumber==2) | (df_val.ServeNumber==0)]['PointWinner'].values
x_val=pd.get_dummies(df_val[(df_val.ServeNumber==2) | (df_val.ServeNumber==0)][Game_test_columns])
labels=x.columns

#undersample = RandomUnderSampler(sampling_strategy=0.9)
#x, y = undersample.fit_resample(x, y)
#print(x.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
x_val=scaler.transform(x_val);
model_xg = XGBClassifier(objective ='binary:logistic',params=random_search.best_params_)


model_xg.fit(X_train, y_train);
y_pred_xg = model_xg.predict(X_test);
scoreXG = accuracy_score(y_pred_xg, y_test)
print(f"Model training score {round(scoreXG,3)*100}")

val_pred=model_xg.predict(x_val)
val_scoreXG = accuracy_score(val_pred, y_val)
evaluation_time(y_val,val_pred,model_xg,x_val)
plot_feature_importance(model_xg.feature_importances_,labels,"XGBooost Revised Second Serve ")
plt.savefig("Pointwinner_S2_V2_Features - XGboost",bbox_inches='tight',dpi=300)

#### Random Forest Grid

In [ ]:
n_estimators=[500,800,1500,2500,5000]
max_features=['auto','accuracy','log2']
max_depth=[10,20,30,40,50]
min_samples_split=[2,5,10,15,20]
min_samples_leaf=[1,2,5,10,15]
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

#### Random forest Hyper parameter tuning

#Data split
X_train, X_test, y_train, y_test = train_test_split(df[Game_test_columns],df['PointWinner'].values, test_size=0.20, random_state=42)

#Feature scaling
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 

#define classifier
RFC = RandomForestClassifier()

#Define search
random_search=RandomizedSearchCV(estimator=RFC,param_distributions=random_grid,n_iter=10,n_jobs=-1,cv=3,verbose=3)

#Run search
start_time = timer(None)
#random_search.fit(X_train,y_train)
timer(start_time) # timing ends here for "start_time" variable

random_search.best_params_

In [ ]:
param={'n_estimators': 1500,
 'min_samples_split': 5,
 'min_samples_leaf': 15,
 'max_features': 'accuracy',
 'max_depth': 10,
 'bootstrap': True}

y=df['PointWinner'].values
x=df[Game_test_columns].values
y_val=df_val['PointWinner'].values
x_val=df_val[Game_test_columns].values
labels=Game_test_columns

#x,y=sample(x,y)

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
x_val=scaler.transform(x_val);


RFC = RandomForestClassifier(n_estimators=1500,min_samples_split=5,min_samples_leaf=15,
                            max_features="log2",max_depth=10,bootstrap=True)
predRFC = RFC.fit(X_train,y_train)
val_pred = predRFC.predict(x_val)
scoreRFC = accuracy_score(val_pred, y_val)
evaluation_time(y_val,val_pred,RFC,x_val)
plot_feature_importance(RFC.feature_importances_,labels,"RF")